Датасет:

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [ ]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
newsgroups_train.target[:100]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6,  0,  1,
        7, 12,  5,  0, 10,  6,  2,  4,  1, 12,  9, 15,  7,  6, 13, 12, 17,
       18, 10,  8, 11,  8, 16,  9,  4,  3,  9,  9,  4,  4,  8, 12, 14,  5,
       15,  2, 13, 17, 11,  7, 10,  2, 14, 12,  5,  4,  6,  7,  0, 11, 16,
        0,  6, 17,  7, 12,  7,  3, 12, 11,  7,  2,  2,  0, 16,  1,  2,  7,
        3,  2,  1, 10, 12, 12, 17, 12,  2,  8,  8, 18,  5,  0,  1])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df=10, max_df=.04)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(binary=True, max_df=0.04, min_df=10,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [ ]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 10299)

In [ ]:
from tqdm import tqdm

def lda(n_dk, n_kw, n_k, z, doc, word, topics, alpha, beta, it):    
    for i in tqdm(range(it)):
        for j in range(len(doc)):
            cur_word = word[j]
            cur_doc = doc[j]
            cur_topic = z[j]
            
            n_dk[cur_doc, cur_topic] -= 1
            n_kw[cur_topic, cur_word] -= 1
            n_k[cur_topic] -= 1
            
            p = (n_dk[cur_doc, :] + alpha) * (n_kw[:, cur_word] + beta[cur_word]) / (n_k + beta.sum())
            z[j] = np.random.choice(np.arange(topics), p=p / p.sum())
            
            n_dk[cur_doc, z[j]] += 1
            n_kw[z[j], cur_word] += 1
            n_k[z[j]] += 1
    return z, n_kw, n_dk, n_k

In [ ]:
z,n_kw, n_dk, n_k=lda(n_dk, n_kw, n_k, z, doc, word, 20, 2*np.ones(20), 2*np.ones(X_train.shape[1]), 50)

100%|██████████| 50/50 [21:13<00:00, 25.47s/it]


In [ ]:
answer = np.argsort(n_kw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in answer[i]:
        matrix[0, j] = 1
    print('Topic {} \t{}'.format(i + 1, '\t'.join(vectorizer.inverse_transform(matrix)[0])))

Topic 1 	article	current	mentioned	posted	sorry	sort	stuff	understand	wait	weeks
Topic 2 	cost	earth	launch	low	major	nasa	orbit	small	space	station
Topic 3 	car	close	comes	curious	game	guess	obviously	original	talking	week
Topic 4 	anti	aren	current	info	light	looks	original	similar	small	type
Topic 5 	anybody	cause	check	feel	guy	lines	money	news	sorry	sound
Topic 6 	anybody	buy	check	correct	interesting	lines	radio	small	source	user
Topic 7 	appreciate	car	clear	goes	higher	ideas	job	large	posting	working
Topic 8 	card	computer	disk	dos	hi	memory	pc	speed	video	works
Topic 9 	came	city	days	home	killed	left	saw	started	took	went
Topic 10 	banks	cadre	chastity	dsl	geb	gordon	intellect	pitt	soon	surrender
Topic 11 	agree	book	btw	fairly	imho	large	nice	oh	small	unless
Topic 12 	00	car	difference	info	instead	news	ok	phone	unless	value
Topic 13 	agree	bible	certainly	christian	christians	claim	jesus	man	religion	saying
Topic 14 	bike	car	difference	left	low	nice	reply	sound	type	worth

Topic 1   медецина


Topic 2 	forsale


Topic 4 	атеизм


Topic 5 	политика(разное)


Topic 6 	forsale


Topic 8   электроника


Topic 9 	политика(оружие)


Topic 10 	экономика


Topic 11 	новости


Topic 12 	автомобили


Topic 13 	религия христианство


Topic 14 	мотоциклы


Topic 15 	политика (америка)


Topic 16 	windows.x


Topic 19 	авто


Topic 20 хоккей
